# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [101]:
import gmaps
import json
import requests
from api_keys import api_key
from api_keys import g_key
import random
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
from scipy.stats import linregress
import os
# Access maps with unique API key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [102]:
cities_pd = pd.read_csv("worldcities.csv")
cities_pd.head(100)


,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id
0,Tokyo,Tokyo,35.6850,139.7514,Japan,JP,JPN,Tōkyō,primary,35676000.0,1392685764
1,New York,New York,40.6943,-73.9249,United States,US,USA,New York,NaN,19354922.0,1840034016
2,Mexico City,Mexico City,19.4424,-99.1310,Mexico,MX,MEX,Ciudad de México,primary,19028000.0,1484247881
3,Mumbai,Mumbai,19.0170,72.8570,India,IN,IND,Mahārāshtra,admin,18978000.0,1356226629
4,São Paulo,Sao Paulo,-23.5587,-46.6250,Brazil,BR,BRA,São Paulo,admin,18845000.0,1076532519
...,...,...,...,...,...,...,...,...,...,...,...
95,Johannesburg,Johannesburg,-26.1700,28.0300,South Africa,ZA,ZAF,Gauteng,admin,3435000.0,1710550792
96,Berlin,Berlin,52.5218,13.4015,Germany,DE,DEU,Berlin,primary,3406000.0,1276451290
97,Algiers,Algiers,36.7631,3.0506,Algeria,DZ,DZA,Alger,primary,3354000.0,1012973369
98,Rome,Rome,41.8960,12.4833,Italy,IT,ITA,Lazio,primary,3339000.0,1380382862


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [103]:
url = "http://api.openweathermap.org/data/2.5/weather?"
cities = cities_pd["city_ascii"]

lat = []
lng = []

temper = []
hum = []
cloud = []
speed = []
cities_list = []


nor_lat = []
nor_hum = []
nor_temper = []
nor_cloud = []
nor_speed = []

sou_lat = []
sou_hum = []
sou_temper = []
sou_cloud = []
sou_speed = []

units = "metric"
impl = "imperial"
query_url = f"{url}appid={api_key}&units={impl}&q="
query_url

'http://api.openweathermap.org/data/2.5/weather?appid=fb969f04e05faa49a7305526d9488b65&units=imperial&q='

In [104]:
for city in cities:
    try:
        response = requests.get(query_url + city).json()
        lat.append(response['coord']['lat'])
        lng.append(response['coord']['lon'])
        temper.append(response['main']['temp'])
        hum.append(response['main']['humidity'])
        speed.append(response['wind']['speed'])
        cloud.append(response['clouds']['all'])
        cities_list.append(city)
        if (response['coord']['lat'] >= 0 ):
            nor_lat.append(response['coord']['lat'])
            nor_temper.append(response['main']['temp'])
            nor_hum.append(response['main']['humidity'])
            nor_speed.append(response['wind']['speed'])
            nor_cloud.append(response['clouds']['all'])            
        else:
            sou_lat.append(response['coord']['lat'])
            sou_temper.append(response['main']['temp'])
            sou_hum.append(response['main']['humidity'])
            sou_speed.append(response['wind']['speed'])
            sou_cloud.append(response['clouds']['all'])  
    except:
        pass
    
weather_dict = {
    "lat": lat,
    "lng": lng,
    "temper": temper,
    "cloud": cloud,
    "speed": speed,
    "hum": hum,
    "city": cities_list
    
}
weather_data = pd.DataFrame(weather_dict)


weather_data.to_csv('cities_with_temper.csv', index=True) 

In [105]:


print(f' the length of lat is {len(weather_dict["lat"])}')
print(f' the length of lng is {len(weather_dict["lng"])}')
print(f' the length of temper is {len(weather_dict["temper"])}')
print(f' the length of cloud is {len(weather_dict["cloud"])}')
print(f' the length of hum is {len(weather_dict["hum"])}')
print(f' the length of speed is {len(weather_dict["speed"])}')
print(f' the length of city is {len(weather_dict["city"])}')

 the length of lat is 1439
 the length of lng is 1439
 the length of temper is 1439
 the length of cloud is 1439
 the length of hum is 1439
 the length of speed is 1439
 the length of city is 1439


In [106]:
# Plot Heatmap
locations = weather_data[["lat", "lng"]]
humidty = weather_data["hum"]
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidty, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [107]:
#perfect weather conditions

per_data = np.arange(len(weather_data))

percnt=0
npercnt=0

perfect_weather_dict = {
    "lat": [],
    "lng": [],
    "temper": [],
    "cloud": [],
    "speed": [],
    "hum": [],
    "city": []
    
}

not_perfect_weather_dict = {
    "lat": [],
    "lng": [],
    "temper": [],
    "cloud": [],
    "speed": [],
    "hum": [],
    "city": []
    
}
per_coordinates = []
not_per_coordinates = []

for x in per_data:
    latlng = ()
    if weather_data["temper"][x] < 80 and weather_data["temper"][x] > 70 and weather_data["speed"][x] < 10 and weather_data["cloud"][x] == 0:
        perfect_weather_dict["lat"].append(weather_data["lat"][x])
        perfect_weather_dict["lng"].append(weather_data["lng"][x])
        perfect_weather_dict["temper"].append(weather_data["temper"][x])
        perfect_weather_dict["cloud"].append(weather_data["cloud"][x])
        perfect_weather_dict["speed"].append(weather_data["speed"][x])
        perfect_weather_dict["hum"].append(weather_data["hum"][x])
        perfect_weather_dict["city"].append(weather_data["city"][x])
        latlng=weather_data["lat"][x],weather_data["lng"][x]
        per_coordinates.append(latlng)
        percnt=percnt+1
    else:
        not_perfect_weather_dict["lat"].append(weather_data["lat"][x])
        not_perfect_weather_dict["lng"].append(weather_data["lng"][x])
        not_perfect_weather_dict["temper"].append(weather_data["temper"][x])
        not_perfect_weather_dict["cloud"].append(weather_data["cloud"][x])
        not_perfect_weather_dict["speed"].append(weather_data["speed"][x])
        not_perfect_weather_dict["hum"].append(weather_data["hum"][x])
        not_perfect_weather_dict["city"].append(weather_data["city"][x])
        latlng=weather_data["lat"][x],weather_data["lng"][x]
        not_per_coordinates.append(latlng)
        npercnt=npercnt+1
        
    
    

perfect_weather_data = pd.DataFrame(perfect_weather_dict)

not_perfect_weather_data = pd.DataFrame(not_perfect_weather_dict)
#not_perfect_weather_data.head()
#len(not_perfect_weather_data)

# Customize the size of the figure
figure_layout = {
    'width': '700px',
    'height': '500px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)
# Assign the marker layer to a variable

markers = gmaps.marker_layer(per_coordinates)
# Add the layer to the map
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…

In [108]:
#not perfect weather conditions
figure_layout = {
    'width': '700px',
    'height': '500px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)
# Assign the marker layer to a variable

markers = gmaps.marker_layer(not_per_coordinates)
# Add the layer to the map
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [110]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

params = {
    "key": g_key,
    "radius": "50"
}

# set up additional columns to hold information
perfect_weather_data['name'] = ""
perfect_weather_data['address'] = ""
perfect_weather_data['country_code'] = ""

dataset= []
locations = perfect_weather_data[["lat", "lng"]]
# use iterrows to iterate through pandas dataframe
for index, row in perfect_weather_data.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

    # get restaurant type from df
    #restr_type = row['hotel']

    # add keyword to params dict
    params['keyword'] = "hotel"
    params['location'] = row["lat"],row["lng"]

    # assemble url and make API request
    #print(f"Retrieving Results for Index {index}: {restr_type}.")
    base_url= base_url+"location="+str(row['lat'])+","+str(row['lng'])+"&radius=5000&keyword=hotel&key="+g_key
    response = requests.get(base_url).json()
    
    # extract results
    results = response['results']
#    dataset.append(results)
    try:
        perfect_weather_data.loc[index, 'name'] = results[0]['name']
        perfect_weather_data.loc[index, 'address'] = results[0]['vicinity']
        perfect_weather_data.loc[index, 'country_code'] = results[0]['plus_code']['compound_code']
    except:
        pass
        
#hotels_nearby = perfect_weather_data["name"].tolist()
#vicinity_nearby = perfect_weather_data["address"].tolist()
#country_nearby = perfect_weather_data["country_code"].tolist()
x_axis = np.arange(len(perfect_weather_data))
fig = gmaps.figure(layout=figure_layout)
markers = gmaps.marker_layer(locations,
info_box_content=[f'<dl><dt>Hotel name :</dt><dd> {perfect_weather_data["name"][ind]} </dd><dt> Address :</dt><dd> {perfect_weather_data["address"][ind]}  </dd><dt>  Country code : </dt><dd>{perfect_weather_data["country_code"][ind]} </dd></dl>' for ind in x_axis])
fig.add_layer(markers)
humidty = perfect_weather_data["hum"]
heat_layer = gmaps.heatmap_layer(locations, weights=humidty, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

fig




Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…

In [111]:
#<dl><dt>Name</dt><dd>{Hotel Name}</dd><dt>City</dt><dd>{City}</dd><dt>Country</dt><dd>{Country}</dd></dl>
#info_box_content=[f'<dl><dt>Hotel name :</dt><dd> {perfect_weather_data["name"][ind]} </dd><dt> and Address :</dt><dd> {perfect_weather_data["address"][ind]}  </dd><dt> and Country code : </dt><dd>{perfect_weather_data["country_code"][ind]} </dd></dl>' for ind in x_axis])


In [112]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
#<dl>
#<dt>Name</dt><dd>{Hotel Name}</dd>
#<dt>City</dt><dd>{City}</dd>
#<dt>Country</dt><dd>{Country}</dd>
#</dl>
#"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
#hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
#locations = hotel_df[["Lat", "Lng"]]

In [113]:
# Add marker layer ontop of heat map


# Display figure
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

params = {
    "key": g_key,
    "radius": "50"
}

# set up additional columns to hold information
perfect_weather_data['name'] = ""
perfect_weather_data['address'] = ""
perfect_weather_data['country_code'] = ""

dataset= []
locations = perfect_weather_data[["lat", "lng"]]
# use iterrows to iterate through pandas dataframe
for index, row in perfect_weather_data.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

    # get restaurant type from df
    #restr_type = row['hotel']

    # add keyword to params dict
    params['keyword'] = "hotel"
    params['location'] = row["lat"],row["lng"]

    # assemble url and make API request
    #print(f"Retrieving Results for Index {index}: {restr_type}.")
    base_url= base_url+"location="+str(row['lat'])+","+str(row['lng'])+"&radius=5000&keyword=hotel&key="+g_key
    response = requests.get(base_url).json()
    
    # extract results
    results = response['results']
#    dataset.append(results)
    try:
        perfect_weather_data.loc[index, 'name'] = results[0]['name']
        perfect_weather_data.loc[index, 'address'] = results[0]['vicinity']
        perfect_weather_data.loc[index, 'country_code'] = results[0]['plus_code']['compound_code']
    except:
        pass
        
#hotels_nearby = perfect_weather_data["name"].tolist()
#vicinity_nearby = perfect_weather_data["address"].tolist()
#country_nearby = perfect_weather_data["country_code"].tolist()
x_axis = np.arange(len(perfect_weather_data))
fig = gmaps.figure(layout=figure_layout)
markers = gmaps.marker_layer(locations,
info_box_content=[f'<dl><dt>Hotel name :</dt><dd> {perfect_weather_data["name"][ind]} </dd><dt> Address :</dt><dd> {perfect_weather_data["address"][ind]}  </dd><dt>  Country code : </dt><dd>{perfect_weather_data["country_code"][ind]} </dd></dl>' for ind in x_axis])
fig.add_layer(markers)
humidty = perfect_weather_data["hum"]
heat_layer = gmaps.heatmap_layer(locations, weights=humidty, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

fig




Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…